# Formula 1 Analysis

For this 1st Project in Preprocessing in Kaggle dediced to chose a database in Kaggle about Formula Historical Data from 1950s to 2023. The reason I chose this dataset was because I followed this sport really close and being able to give a deep look into some statistics would be interesting due to all the data that is available.

Dataset belongs to Volani which made a solid work with the web scraping through a database of F1 Championships standinds, points, races and invidual stats from Formula 1 drivers. However this is not a good dataset yet since it has some issues with naming of drivers and some data it has missing fields. 

# Data Inspection

For this part we will have to address the following data:

* Circuit 
    - Contains 9 Columms
    - Integers: circuitID, altitude
    - Strings: circuitref, name, location, country
    - Float: longitude, latitude
    - No missing values, URL have no purpose in the data pipeline
    
* Constructor Results 
    - Contains 5 Columms
    - Integers: constructorResults, raceid, constructor id, points
    - Status columm have a lot of duplicates, will be removed later 
    
* Constructor Standings 
    - Contains 7 Columms
    - Integers: constructorsStandingid, raceid, constructorid, points, position, wins
    - positionInText is a duplicate of position in a string format, we remove it this
* Constructors
    - Contains 5 Columms
    - Integers: constructorsid, 
    - Strings: constructorRef, name, nationality
    - URL have no purpose in the data pipeline
* driver_standings
    - Contains 7 Columms
    - Integers: standingsid, raceid, driverid, points, position, wins
    - positionInText is a duplicate of position in a string format, we remove it this
* drivers
    - Contains 9 Columms
    - Integers: number, driverid, points, position, wins
    - Strings: driverRef, surname, nationality
    - dob will be removed since in the sampling data we won't require this
    - URL have no purpose in the data pipeline
* lap_times
    - Contains 6 Columms
    - Integers: raceid, lap, position, time
    - Time in milliseconds will be reduced to ms abbreviation
    - There's 4 columms with 500k rows each, we will compress them later into the sampling for visualization
* pit_stops 
    - Contains 7 Columms
    - Integers: raceid, lap, position, time, raceid, stop, duration
    - Time in milliseconds will be reduced to ms abbreviation
* qualifying 
    - Contains 9 Columms
    - Integers: qualifyid, raceid, driverid, constructorid, number, position, q1, q2, q3
    - Since Q1 - Q3 is a elimination format meaning that there will be missing data, will be equalize missing data as DNF or in the Number termination as 2 minutes to their duration of the round they were eliminated.
    - URL have no purpose in the data pipeline
* races 
    - Contains 18 Columms
    - Integers: raceid, year, round, circuitid, time, fp1date, fp1time, fp2date, fp2time, fp3date, fp3time, qualidate, qualitime, sprintdate, sprinttime
    - String: Name, date,
* results
    - Contains 18 Columms
    - Integers: resultid, raceid, driverid, constructorid, number, grid, position, position order, points, laps, time
    - positionInText is a duplicate of position in a string format, we remove it this
* seasons
    - Year is the only column that will be taken into account for this csv
* sprint_results
    - Data recording started 2021, is already compressed and contains no missing values



# Data Preprocessing (classification)

Akhil Reddy, notebook expert in Kaggle. provided really good documentation to understand how can we assess this information we have. Using pandas we will be able to showcase some charts for this visualizations I will be providing for the F1 Data. 

Not all data is relevant therefore I must do a visual inspection of what would I need to consider, a lot of the CSVs contain data that is related universally and must be used in order to group it and pipeline it to the correct figure we want to show. The focus point of this is to showcase the perfomance of the constructors, which are the cars manufacturers and teams in Formula 1. In addition we will showcase interesting geographical data such as nationality of drivers that have won, teams by nationality, constructors throughout the years.

# Data Cleaning 

For the data cleaning I did 3 steps

- 1. Missing Fields

For certain Qualifiers which are rounds of elimination were drivers after being eliminated do not register lap time, meaning that this would be having a null value in their record since the drivers that are still competing in the futher round will have a record on that. 

To fix this issue in the CSV, we are considering only the time of their Qualifying the got eliminated, meaning that we reduced the missing fields to duplicating the "replace_all" inside the csv with "if-else" if the missing there's a missing field then it will take the value previous to the missing field, since everyone has a qualifying 1 value, there will be a value duplicate to all qualifiers

Would duplicating the value hurt the statistics? not for our visualization, we are focusing on tracking the quantity the participate in rather than perfomance per driver, although we are including the constructor permonance, lap time is not consider as one of the variables that will influence our chart. 

- 2. Driver's Name

The Umlaut, is a the dots over the letter that is well seen in names and places in across nordic countries. In formula 1, we have drivers all over the world, the main issue with our drivers is that their symbols are misrespresented in the CSVs. For that same reason I went to only the Drivers CSV and made sure that Nordic drivers were having their umlaut typed in a way its looks well represented. 

- 3. Compressing Data 

We possess a lot of data such that we have 2 Million rows in 4 CSVs, for this before visualization we will merge the rows and group them so there's no unnecessary use of unique information. 

# Data Visualization 

For the Visualization of the data, we will use different libraries to plot in a graph and show historgrams. Next step will be importing them. 


In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import datetime


Once libraries are set to use, we will require to provide a way to read the CSVs, in this case I went ahead and upload them in github repo: https://github.com/Scamp24/f1data and from here, using the rawgithub data format is an easier way to have my data hosted on that end instead of reading from local machine. An alternative could have been using the same localhost of this notebook, but for this project github repositories was preffered.


* Race circuits stats

In [2]:
circuits = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/circuits.csv')
laptimes = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/lap_times.csv')
pitstops = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/pit_stops.csv')
seasons = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/qualifying.csv',parse_dates=['year'])
status = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/qualifying.csv')

* Constructor Stats

In [3]:
constructor_standings = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/constructor_standings.csv')
constructors = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/constructors.csv')
driver_standings = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/driver_standings.csv')
drivers = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/drivers.csv')

* Results Stats (Compilation)

In [6]:
races = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/races.csv',parse_dates=['year'])
constructor_results = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/constructor_results.csv')
results = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/results.csv')
qualifying = pd.read_csv('https://raw.githubusercontent.com/Scamp24/f1data/main/qualifying.csv')

The next thing that follows is making sure we are merging data, remember we have a lot of data spreadout in a lot of CSVs. This is the time where we come to merge and make sure that we can compress the data even futher to produce an output on to what we want

In [7]:
team = constructors.merge(results,on='constructorId',how = 'left')

We are creating the variable 'team' to store the merge in constructors CSV, with the parameters set to look for a 'constructorsId' in result CSV and this will merge the dataframes of constructors and races.


In [8]:
best = team[['name','points','raceId']]
best = best.groupby('name')['raceId'].nunique().sort_values(ascending=False).reset_index(name = 'races')
best = best[best['races'] >= 100]
best.head() 

,name,races
0,Ferrari,1066
1,McLaren,895
2,Williams,809
3,Tyrrell,433
4,Renault,403


This table shows the Top 5 teams with most races raced. Isn't that incredible? It is but something better is to find which team is better at each race, perhaps the results might be different? 

To find this we have to calculate the points a team earns per race. Points are the sum between the 2 drivers of the same constructor, this will provide how well a team performs race to race. 

Isn't the goal looking for a subset of the data? yes, but we are ensuring we are showing a set amount of teams of Formula 1 throughout history, while it seems counterproductive to use all the data since 1950s, we can ensure that we take into account all the historial data but we will sample only the top 10-15 teams that perfomed throughout history. 

In [9]:
func = lambda x: x.points.sum()/x.raceId.nunique()
data = team[team['name'].isin(best.name)].groupby('name').apply(func).sort_values(ascending=False).reset_index(name = 'points_per_race')
data.head(10)

,name,points_per_race
0,Mercedes,25.299081
1,Red Bull,18.922222
2,Ferrari,9.660666
3,McLaren,6.972626
4,Force India,5.179245
5,Williams,4.461063
6,Renault,4.409429
7,Benetton,3.313462
8,BRM,2.584135
9,Team Lotus,2.518987


Now we have a clear view of which teams are the most likely to be successful per race across F1. Teams are presented as a table but we much rather would like to see the data in another way. For this instance we will use a histogram to represent what's the visual difference when refer to data rather than just the difference between numbers. 



In [14]:
fig = go.Figure(
    data=[go.Bar(x = data.name, y=data['points_per_race'])],
    layout_title_text="Constructor's Points per Race"
    
)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=1)))
fig.show()


The difference between a table and histogram is more noticeable, we can clearly see there are 2 dominants teams in an average race of F1. For instance we see Mercedes and Redbull and the biggest scoring teams, as far as we go in the graph, more and more teams seem more incosistent to their point scoring strategies. 

For more in-depth statistics we can realize some facts about teams in Formula 1:

* The mean points per race across all teams in this historical Formula 1 data is approximately 8.891 points.

* The upper quartile (Q3) represents the 75th percentile of the data. In this case, the upper quartile for points per race is around 18.058 points. This means that 75% of the teams had points per race less than or equal to 18.058.

* The lower quartile (Q1) represents the 25th percentile of the data. In this case, the lower quartile for points per race is approximately 4.409 points. This implies that 25% of the teams had points per race less than or equal to 4.409.

* The data shows a significant variation in points per race across the Formula 1 teams. Mercedes has the highest average points per race at 25.299, indicating their strong performance historically. On the other hand, Team Lotus and BRM have much lower average points per race, suggesting they may not have been as competitive throughout the historical period covered by the data.



Now we move on the other sample visualization that is the "Constructors Historic Points", For this we only get a subset of the data, Top 10 Constructors in historic points. 

In [11]:
historic_points = team.groupby('name').agg({'points':'sum'}).sort_values('points',ascending=False).reset_index().head(10)
historic_points

,name,points
0,Ferrari,10298.27
1,Mercedes,7159.64
2,Red Bull,6812.00
3,McLaren,6240.50
4,Williams,3609.00
5,Renault,1777.00
6,Force India,1098.00
7,Team Lotus,995.00
8,Benetton,861.50
9,Tyrrell,711.00


What we just did was creating a variable called historic points which will group the teams and their sum of points historically and sort their points in descending order. 

We would to simplify data and showcase it in a histogram as we did with the average points per race per team. 


In [13]:
fig = go.Figure(
    data=[go.Bar(x = historic_points.name, y=historic_points['points'])],
    layout_title_text="Constructor's Historic Points"
)
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=1)))
fig.show()

Same case happened like the previous scenario, we compress the data to only the top 10 and we checked that is easier to verify which teams has been more dominant. 

In-depth statistics:

* The mean total points across all teams in this data is approximately 4464.79 points.

* The upper quartile (Q3) represents the 75th percentile of the data. In this case, the upper quartile for total points is around 6214.75 points. This means that 75% of the teams had total points less than or equal to 6214.75.

* The lower quartile (Q1) represents the 25th percentile of the data. In this case, the lower quartile for total points is approximately 995.00 points. This implies that 25% of the teams had total points less than or equal to 995.00.

* Mercedes and Red Bull follow Ferrari with total points of 7159.64 and 6812.00, respectively. These three teams have consistently been among the top performers.

* There is a significant drop in total points as you move down the table, with Williams, Renault, and Force India having total points in the thousands but considerably lower than the top-performing teams.


There's other ways we could see information, such as pie charts. For instance we can do the following scenarios:

* What's percentage of driver's nation across history?
* What's What's the percentage of driver's nationality that have been world champions?
* What's the likeness from being any nation and being a champion?

This kind of questions could be solved by looking at percentages of over the data, but once again, tables are not feasible to understand the magnitude of the difference between nations. For this scenario pie charts will be applied to showcase a better understanding.



In [15]:
driver_nationality = drivers.groupby('nationality')['nationality'].count().sort_values(ascending = False).reset_index(name = 'number of drivers')
fig = go.Figure(data=[go.Pie(labels=driver_nationality.nationality.head(10), values=driver_nationality['number of drivers'])])
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    title="Historical Driver Nationality Distribution since 1950")
fig.show()

This pie chart allows us to check on what driver nationality is the most common so far:

* British, American and Italian are the most Common drivers in formula 1 History

* South america has minor representation of the graph with only argentines and brazilians being the most common from this continent

* There's no representation no significant representation from Oceania and Asia 



Is the same nationality dominance in the Champion's pie chart? for that we will have to merge a lot of data that have results involved in. 



In [16]:
driver_position = drivers.merge(driver_standings,left_on='driverid',right_on='driverId',how = 'left')
driver_position = driver_position.merge(races,on = 'raceId',how = 'left')

We are creating the Driver position variable to be able to merge their points and position from their race results.

In [17]:
champions = driver_position.groupby(['nationality','year','surname'])[['points','wins']
                                            ].max().sort_values('points',ascending = False).reset_index()
champions.drop_duplicates(subset=['year'], inplace=True)

However not every driver is a Champion, a champion is the one who scores the most points in a season. The sum of all races points, from the same year and track the nationality. This will be able showcase only drivers that are being champions. With this we have only 72 Champions out of the 1000+ drivers. Could anyone be two-time? three time? duplicates? the 72 Champions are not unique some of them are even more successful to win more than once. However we are counting only the instance where the actual country won, meaning that a driver could win more than once and represent more than once that country which will be a variable towards our pie chart. 


In [18]:
champion_nations = champions.nationality.value_counts().to_frame()
fig = go.Figure(data=[go.Pie(labels=champion_nations.index, values=champion_nations['nationality'])])
fig.update_traces(textfont_size=20,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout(
    title="Distribution of Historic Champions by Nation")
fig.show()

We see a interesting ratio between British drivers, it seems like you are more likely to be a champion in you are born in England, chances are approximately 1/4. 

Another interesting fact we see is that the American drivers seems like they haven't been a champion in their entire history. Making this a complete 0:1 ratio for Americans



# Data Sampling

For all the instances in the project, we haved done some techniques to be able to keep information relevant and useful. A few of the techniques we had involve:

* Clustering 
The identifiers in all CSVs allow us to merge data and creating new sources for information we found similarities and we tied the results to all the teams and constructors. 

* Sampling 
We took a large amount of data and represent a small subset, for this case we didn not consider all countries and all possible teams. In some scenarios we reduced them to Top 10 or Top 5

* Missing data imputation 

We made sure that the pre-processed data was full and complete before going into a process of pipelining and visualization. 



# Conclusion

In formula 1 throughout its 74 years, Ferrari, Mercededs and Redbull has been the best and most dominant constructors. In drivers we have seen that the most successful and most representative is british drivers. 

